<a href="https://colab.research.google.com/github/ricoen/google-ee-data-extraction/blob/main/ee_data_extraction_aoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine Data Extract

Author [@ricoen](https://github.com/ricoen)<br>
Please cite me if you use this code :)

In [ ]:
#Import Earth Engine
import ee

In [ ]:
#Authenticate EE with your account
ee.Authenticate()

In [ ]:
#Initialize EE
ee.Initialize()

In [ ]:
#Set the dataset from EE Image Collection
dataset = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")

In [ ]:
#Set the dataset taken time
taken = dataset.filterDate("2022-07-01", "2022-07-31")

In [ ]:
#Get the first raster for visualize the data
first_raster = ee.Image(taken.first())

In [ ]:
#Get info from the selected raster
img = first_raster

img.getInfo()

In [ ]:
#Get the temperature value from T band
temperature = img.select("Tair_f_inst")

In [ ]:
#Set the AOI (Area of Interest)
aoi = ee.Geometry.Polygon([
          [
            [
              112.62535572052002,
              -7.988390117845741
            ],
            [
              112.64822959899902,
              -7.988390117845741
            ],
            [
              112.64822959899902,
              -7.968202639589894
            ],
            [
              112.62535572052002,
              -7.968202639589894
            ],
            [
              112.62535572052002,
              -7.988390117845741
            ]
          ]
        ], None, False)

#poi = ee.Geometry.Point([112.644105, -7.9514187])

In [ ]:
# Reduce the raster
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), aoi)
    return image.set(dict)

In [ ]:
#Calculate the raster property
withMean = taken.map(setProperty)

In [ ]:
import pandas as pd

#Convert temperature unit (Kelvin) to Celcius 
t_avg = withMean.aggregate_array("Tair_f_inst").getInfo()
t_c = [x - 273 for x in t_avg]
t_c = pd.Series(t_c).round(decimals=0)

#print(t_c)

In [ ]:
#Get the date
from datetime import datetime as dt

time = withMean.aggregate_array("system:time_start").getInfo()
#Convert from miliseconds to true time (local time with 24-H format)
date = [x // 1000 for x in time]
captured = [dt.utcfromtimestamp(i).strftime("%c GMT") for i in date]
captured = pd.Series(captured)
converted_date = pd.to_datetime(captured).dt.tz_convert("Asia/Jakarta")

# print(captured)

In [ ]:
#Just look inside the array
t_c

In [ ]:
#Just look inside the array
converted_date

In [ ]:
#Visualize The Data

#Convert from numpy array to Pandas Series
date = pd.Series(converted_date)
day = date.dt.day_name()
temperature = pd.Series(t_c)
#Concatnate the columns of Date and Temperature
df = pd.concat([day, date, temperature], axis=1)
df.columns = ["day", "date", "temperature"]

df

In [ ]:
#Visualize the data with Plotly
import plotly.express as px

#Define x and y values
fig = px.line(df, x="date", y="temperature", 
              title="Air Temperature on Malang City - GLDAS-2.1 (NASA)")
#Show the figure
fig.show()